# Enforcing fairnes with AIF360

<a href="https://colab.research.google.com/drive/1ymhBHZGdplUU2l0PcJhNQRmt47z83wZY" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).

Machine Learning Fairness is an area of research focusing on ensuring that machine learning algorithms are fair and unbiased without reinforcing or amplifying existing societal inequalities. The importance of machine learning fairness lies in the fact that algorithms can sometimes perpetuate or amplify biases in the training data or the algorithm design. This can lead to discriminatory outcomes that disadvantage certain groups based on their race, gender, or other protected characteristics. Thus, ensuring fairness in machine learning algorithms is essential to build trust and accountability.

<img src="https://miro.medium.com/max/1060/1*cc8OWxqKFXje4d_1eYrQkg.jpeg" width="600"/>

Source: [Moritz Hardt](https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb/).

In this notebook, we will be looking at some discrimination happening on the grounds of Hogwarts. Since Hogwarts does not exist, we will create a fake dataset to represent the admissions history in this prestigious school of magic. The dataset will be made with a bias against an unprivileged group for this tutorial. 🐍 🧙 🦁

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Create the unprivileged group
unpriv = pd.DataFrame({'House': ['Slytherin' for i in range(0, 400)],  # 400 unprivileged samples
                       'Score': np.random.normal(5.5, 0.6, 400),
                       'output': np.random.choice([0, 1], size=(400), p=[0.5, 0.5])})

# Create the privileged group
priv = pd.DataFrame({'House': ['Gryffindor' for i in range(0, 1000)],  # 1000 unprivileged samples
                     'Score': np.random.normal(6.5, 0.4, 1000),
                     'output': np.random.choice([0, 1], size=(1000), p=[0.2, 0.8])})

group_labels = ['Slytherin', 'Gryffindor']
colors = ['green', 'red']

# Plot their respective distributions in grades
fig = ff.create_distplot([unpriv['Score'], priv['Score']], group_labels, bin_size=.1,
                         curve_type='normal',
                         colors=colors)
fig.update_layout(title_text='The Hogwarts First Year Entrance Exam',
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()


groups = pd.concat([priv, unpriv]).reset_index(drop=True)

for i in np.arange(len(groups)):
    if (groups.at[i, 'Score'] < 5.3):
        groups.at[i, 'output'] = np.random.choice([0, 1], p=[0.7, 0.3])
    elif (groups.at[i, 'Score'] > 6.2):
        groups.at[i, 'output'] = np.random.choice([0, 1], p=[0.2, 0.8])

gf = groups[groups['House'] == 'Gryffindor']
sl = groups[groups['House'] == 'Slytherin']

# Plot their rate of admission
fig = go.Figure([go.Bar(x=['Slytherin', 'Gryffindor'], y=[
                sl['output'].sum(), gf['output'].sum()], marker_color=colors)])

fig.update_layout(title_text='The Hogwarts First Year Entrance Exam Total Approval',
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

Done! We have a biased history of admissions. The AI that controls the selector hat seems to be rejecting the entry, much more often, to Slytherin students than, for example, the "_privileged house_" Gryffindor. 🐍 🦁

Let us train this year's version of the hat with our historically biased dataset. We will remove the `House` feature to blind our algorithm to "_the house of the candidate_" hoping that the veil of ignorance might help our Slytherin comrades.

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import plotly.express as px

# Set a seed for reproducibility purposes
seed = 42

groups_array = groups.drop('House', axis=1).values

# Split the dataset into train/test
train, test = train_test_split(
       groups, stratify=groups['House'],
       test_size=0.2,
       random_state=seed
)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

# Initiate a `LogisticRegression` and fit it to the dataset
lr = LogisticRegression(random_state=seed)
lr.fit(train[['Score']], train['output'])

# Evaluate the `LogisticRegression` classifier
score = lr.score(test[['Score']], test['output'])
preds = lr.predict(test[['Score']])
print(f'Accuracy: ' + '{:.2f}'.format(score * 100) + ' %')

# Plot results as a confusion matrix
matrix = confusion_matrix(test['output'], preds)

fig = px.imshow(matrix,
                labels=dict(x="Predicted", y="True label"),
                x=['Accepted', 'Not accepted'],
                y=['Accepted', 'Not accepted'],
                text_auto=True
                )

fig.update_xaxes(side='top')

fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')

fig.show()

Accuracy: 72.50 %


According to this matrix, Hogwarts is a tough school. The algorithm we train err more often in denying entry to students who should have gotten in (especially people from the unprivileged group)... But how can we quantify how much this algorithm discriminates against certain groups in favor of others? We can use `fairness metrics`.

### Statistical Fairness Metrics

_Fairness metrics are a set of measures that enable you to detect the presence of bias in your model._ For a full review of the most prominent definitions of fairness in algorithmic classification, we recommend [_Fairness Definitions Explained_](https://fairware.cs.umass.edu/papers/Verma.pdf). Below, you will see how to apply these definitions.

Since all these definitons are a way to compare the rates of correct/incorrect predictions, it is worth to remember what is a confusion matrix. A confusion matrix is a table often used to describe the performance of a classification model (or "classifier") on a set of test data for which the actual values are known.

![confusion-matrix](https://upload.wikimedia.org/wikipedia/commons/3/32/Binary_confusion_matrix.jpg)

Where:

- $TP$: True Positive (_correct positive classification_, e.g., should get in and got in)
- $FP$: False Positive (_incorrect positive classification_, e.g., should not get in but got in)
- $TN$: True Negative (_correct negative classification_, e.g., should not get in and did not got in)
- $FN$: False Negative (_incorrect negative classification_, e.g., should get in but did not got in)

With these values, there are several ways to assess how biased an algorithm may be by comparing how certain combinations of these scores change about specific groups.

> **Note: In the literature, it is common to refer to these groups, in the context of a fairness analysis, as `privileged` or `unprivileged`.**

### Common Statistical Fairness Metrics

- **Statistical Parity Ratio:** Statistical Parity Ratio compares the proportion of members of a given group that were classified for the positive class (i.e., correctly or not, a.k.a., TP and FP) to another group (privileged versus unprivileged).

$$\frac{\text{Statistical Parity}_{\text{ unprivileged}}}{\text{Statistical Parity}_{\text{ privileged}}} = \frac{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})_{\text{unprivileged}}}{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})_{\text{privileged}}}$$

- **Equal Opportunity Ratio:** Equal Opportunity ratio compares the true positive rate (i.e., TPR, a.k.a., _Sensitivity/Recall_) of different groups (privileged versus unprivileged).

$$\frac{\text{TPR}_{\text{ unprivileged}}}{\text{TPR}_{\text{ privileged}}} = \frac{(\frac{TP}{TP+FN})_{\text{unprivileged}}}{(\frac{TP}{TP+FN})_{\text{privileged}}}$$

- **Predictive Parity Ratio:** Predictive Parity Ratio compares the positive predictive value (i.e., PPV, a.k.a., _Precision_) of different groups (privileged versus unprivileged).

$$\frac{\text{PPV}_{\text{ unprivileged}}}{\text{PPV}_{\text{ privileged}}} = \frac{(\frac{TP}{TP+FP})_{\text{unprivileged}}}{(\frac{TP}{TP+FP})_{\text{privileged}}}$$

- **Predictive Equality Ratio:** Predictive Equality Ratio compares the false positive rate (FPR, a.k.a., _fall-out/false alarm ratio_) of different groups (privileged versus unprivileged).

$$\frac{\text{FPR}_{\text{ unprivileged}}}{\text{FPR}_{\text{ privileged}}} = \frac{(\frac{FP}{FP+TN})_{\text{unprivileged}}}{(\frac{FP}{FP+TN})_{\text{privileged}}}$$

- **Accuracy Equality Ratio:** Accuracy Equality Ratio compares the proportion of members of a given group that were correctly classified (i.e., _accuracy_) to another group (privileged versus unprivileged).

$$\frac{\text{Accuracy}_{\text{ unprivileged}}}{\text{Accuracy}_{\text{ privileged}}} = \frac{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})_{\text{unprivileged}}}{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})_{\text{privileged}}}$$

- **Equalized Odds:** Equalized Odds it is the most restrictive concept of ML Fairness. This criteria is only satisfied if both groups (privileged versus unprivileged) have equal TPR and FPR.

$$\frac{\text{TPR}_{\text{ unprivileged}}}{\text{TPR}_{\text{ privileged}}} = 1 \;\land\;\frac{\text{FPR}_{\text{ unprivileged}}}{\text{FPR}_{\text{ privileged}}} = 1$$

Let us now  implement a function that calculates all these metrics.

In [3]:
from IPython.display import Markdown

def calc_fair(model, DataFrame, protected_atributte, group_priv, group_unpriv, label):
    """
    The function calc_fair computes several fairness metrics for a given machine
    learning model on a test set DataFrame. The fairness metrics calculated include
    statistical parity ratio, true positive rate, positive predictive value,
    false positive rate, accuracy, equal opportunity ratio, predictive parity
    ratio, predictive equality ratio, and accuracy equality ratio. The function
    takes in the following arguments:

    Args:
    --------
        - model: The trained machine learning model to evaluate fairness on.
        - DataFrame: The test set data used to evaluate the model.
        - protected_attribute: The name of the protected attribute in the DataFrame.
        - group_priv: The value of the protected attribute for the privileged group.
        - group_unpriv: The value of the protected attribute for the unprivileged group.
        - label: The name of the column in the DataFrame that contains the ground truth labels.

    Returns:
    --------
    The function returns a dictionary containing the fairness metric names and their
    corresponding scores. The scores are rounded to two decimal places. Additionally,
    the function returns the equalized odds as a string.
    """
    test_set = DataFrame.copy()

    test_set_priv_labels, test_set_priv = list(test_set[test_set[protected_atributte] == group_priv][label]), test_set[test_set[protected_atributte] == group_priv].drop([label, protected_atributte], axis = 1)
    test_set_unpriv_labels, test_set_unpriv = list(test_set[test_set[protected_atributte] == group_unpriv][label]), test_set[test_set[protected_atributte] == group_unpriv].drop([label, protected_atributte], axis = 1)

    preds_priv = model.predict(test_set_priv)
    preds_unpriv = model.predict(test_set_unpriv)

    TN_PV, FP_PV, FN_PV, TP_PV = confusion_matrix(test_set_priv_labels, preds_priv).ravel()
    TN_UPV, FP_UPV, FN_UPV, TP_UPV = confusion_matrix(test_set_unpriv_labels, preds_unpriv).ravel()

    statistical_parity_priv = (TP_PV + FP_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # STATISTICAL PARITY RATIO
    statistical_parity_unpriv = (TP_UPV + FP_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # STATISTICAL PARITY RATIO
    equal_oportunity_priv = TP_PV / (TP_PV+FN_PV)  # TRUE POSITIVE RATIO
    equal_oportunity_unpriv = TP_UPV / (TP_UPV+FN_UPV)  # TRUE POSITIVE RATIO
    predictive_parity_priv = TP_PV/(TP_PV + FP_PV)  # POSITIVE PREDICTIVE VALUE
    predictive_parity_unpriv = TP_UPV/(TP_UPV + FP_UPV)  # POSITIVE PREDICTIVE VALUE
    predictive_equality_priv = FP_PV / (FP_PV+TN_PV)  # FALSE POSITIVE RATE
    predictive_equality_unpriv = FP_UPV / (FP_UPV+TN_UPV)  # FALSE POSITIVE RATE
    accuracy_equality_priv = (TP_PV + TN_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # ACCURACY EQUALITY RATIO
    accuracy_equality_unpriv = (TP_UPV + TN_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # ACCURACY EQUALITY RATIO

    if statistical_parity_priv >= statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_unpriv/statistical_parity_priv
    elif statistical_parity_priv < statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_priv/statistical_parity_unpriv

    if equal_oportunity_priv >= equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_unpriv/equal_oportunity_priv
    elif equal_oportunity_priv < equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_priv/equal_oportunity_unpriv

    if predictive_parity_priv >= predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_unpriv/predictive_parity_priv
    elif predictive_parity_priv < predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_priv/predictive_parity_unpriv

    if predictive_equality_priv >= predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_unpriv/predictive_equality_priv
    elif predictive_equality_priv < predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_priv/predictive_equality_unpriv

    if accuracy_equality_priv >= accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_unpriv/accuracy_equality_priv
    elif accuracy_equality_priv < accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_priv/accuracy_equality_unpriv

    data = {'Fairness Metrics': ['Chance of receiving the positive class - privileged',
                                'Chance of receiving the positive class - unprivileged',
                                'Statistical Parity Ratio (SPR)',
                                'True Positive Rate - privileged',
                                'True Positive Rate - unprivileged',
                                'Equal Opportunity Ratio (EOR)',
                                'Positive Predictive Value - privileged',
                                'Positive Predictive Value - unprivileged',
                                'Predictive Parity Ratio (PPR)',
                                'False Positive Rate - privileged',
                                'False Positive Rate - unprivileged',
                                'Predictive Equality Ratio (PER)',
                                'Accuracy - privileged',
                                'Accuracy - unprivileged',
                                'Accuracy Equality Ratio (AER)',
                                'Equalized Odds'],
            'Scores': [round(statistical_parity_priv, 2),
                        round(statistical_parity_unpriv, 2),
                        round(statistical_parity_ratio,2),
                        round(equal_oportunity_priv, 2),
                        round(equal_oportunity_unpriv, 2),
                        round(equal_oportunity_ratio, 2),
                        round(predictive_parity_priv,2),
                        round(predictive_parity_unpriv,2),
                        round(predictive_parity_ratio,2),
                        round(predictive_equality_priv,2),
                        round(predictive_equality_unpriv,2),
                        round(predictive_equality_ratio,2),
                        round(accuracy_equality_priv,2),
                        round(accuracy_equality_unpriv,2),
                        round(accuracy_equality_ratio,2),
                        f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)}. FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}']
            }
    return pd.DataFrame(data).set_index('Fairness Metrics')

fairness_df = calc_fair(lr, test, 'House', 'Gryffindor', 'Slytherin', 'output')
display(Markdown(fairness_df.to_markdown()))

| Fairness Metrics                                      | Scores                             |
|:------------------------------------------------------|:-----------------------------------|
| Chance of receiving the positive class - privileged   | 1.0                                |
| Chance of receiving the positive class - unprivileged | 0.52                               |
| Statistical Parity Ratio (SPR)                        | 0.52                               |
| True Positive Rate - privileged                       | 1.0                                |
| True Positive Rate - unprivileged                     | 0.69                               |
| Equal Opportunity Ratio (EOR)                         | 0.69                               |
| Positive Predictive Value - privileged                | 0.77                               |
| Positive Predictive Value - unprivileged              | 0.48                               |
| Predictive Parity Ratio (PPR)                         | 0.62                               |
| False Positive Rate - privileged                      | 1.0                                |
| False Positive Rate - unprivileged                    | 0.43                               |
| Predictive Equality Ratio (PER)                       | 0.43                               |
| Accuracy - privileged                                 | 0.77                               |
| Accuracy - unprivileged                               | 0.61                               |
| Accuracy Equality Ratio (AER)                         | 0.8                                |
| Equalized Odds                                        | TPR: 1.0 vs 0.69. FPR: 1.0 vs 0.43 |

Fairness metrics can help us determine if our model discriminates according to any particular definition of fairness. However, as an [impossibility theorem](https://arxiv.org/abs/2007.06024), some Fairness metrics (_Statistical Parity_, _Equalized Odds_, _Predictive Parity_) are incompatible and cannot be completely satisfied simultaneously. Thus, the choice of which metric to use must be made according to the context of an application (i.e., benefit awarding, medical diagnosis, etc.).

After diagnosing a bias, how can we fix it? **One possible solution is using the [AI Fairness 360](https://www.ibm.com/opensource/open/projects/ai-fairness-360/) toolkit.** ⚖️🛠️

AI Fairness 360 is an extensible open-source toolkit that can help you examine, report, and mitigate discrimination and bias in machine learning models throughout the AI application lifecycle. We invite you to use and improve it.

In this notebook, we are implementing three levels of possible repair (`1.0, 0.8, 0.7`) by a technique proposed by [Rachel K. E. Bellamy](https://arxiv.org/search/cs?searchtype=author&query=Bellamy%2C+R+K+E) and collaborators: the **_[Disparate Impact Remover](https://arxiv.org/abs/1810.01943)_.**

We will first change the "House" features to numerical representations (`0,1`). Then we will create a train/test `BinaryLabelDataset`, where we can specify factors like `favorable_label` (`1` = getting into Hogwarts), `protected_attribute_names` (`House`), and `unprivileged_protected_attributes` (`not_gryffindors`).

Then, we will `correct` (a.k.a. disturb the dataset in a way to maximize fairness) the created binary dataset with the `DisparateImpactRemover`, using different levels of repair. Optimizing for fairness may lead the model to regions where accuracy will be penalized. Thus, the higher the level of repair, the more accuracy we will lose.

In [6]:
# First, we install the `aif360` and `BlackBoxAuditing` libraries
!pip install aif360 BlackBoxAuditing -q

from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover

mapping = {'Slytherin': '0', 'Gryffindor': '1'}

train['House'].replace(mapping, inplace=True)
test['House'].replace(mapping, inplace=True)

def process_dataset(dataset, repair_level):
    BLD = BinaryLabelDataset(favorable_label='1',
                             unfavorable_label='0',
                             df=dataset,
                             label_names=['output'],
                             protected_attribute_names=['House'],
                             unprivileged_protected_attributes=['0'])

    di = DisparateImpactRemover(repair_level=repair_level)
    repaired_data = di.fit_transform(BLD)

    repaired_pd = pd.DataFrame(np.hstack(
        [repaired_data.features, repaired_data.labels]), columns=['House', 'Score', 'labels'])
    repaired_pd['House'].replace({0.0: 'Slytherin', 1.0: 'Gryffindor'}, inplace=True)

    return repaired_pd

repaired_train = process_dataset(train, repair_level=1.0)
repaired_test = process_dataset(test, repair_level=1.0)

repaired_train_2 = process_dataset(train, repair_level=0.8)
repaired_test_2 = process_dataset(test, repair_level=0.8)

repaired_train_3 = process_dataset(train, repair_level=0.7)
repaired_test_3 = process_dataset(test, repair_level=0.7)



Let us now see how the `DisparateImpactRemover` changed the distribution of our restored datasets.

In [8]:
import plotly.figure_factory as ff

# Define tables for repaired datasets (repair_level=1.0)
unpriv_rp = repaired_train_pd.loc[repaired_train_pd['House'] == 'Slytherin']
priv_rp = repaired_train_pd.loc[repaired_train_pd['House'] == 'Gryffindor']

# Define tables for repaired datasets (repair_level=0.8)
unpriv_rp_2 = repaired_train_2_pd.loc[repaired_train_2_pd['House'] == 'Slytherin']
priv_rp_2 = repaired_train_2_pd.loc[repaired_train_2_pd['House'] == 'Gryffindor']

# Define tables for repaired datasets (repair_level=0.7)
unpriv_rp_3 = repaired_train_3_pd.loc[repaired_train_3_pd['House'] == 'Slytherin']
priv_rp_3 = repaired_train_3_pd.loc[repaired_train_3_pd['House'] == 'Gryffindor']


def create_distplot_and_show(data, title, template='plotly_dark'):
    fig = ff.create_distplot(data, group_labels, bin_size=.1,
                             curve_type='normal', colors=colors)
    fig.update_layout(title_text=title,
                      template=template,
                      paper_bgcolor='rgba(0, 0, 0, 0)',
                      plot_bgcolor='rgba(0, 0, 0, 0)')
    fig.show()

# Original Data
create_distplot_and_show([unpriv['Score'], priv['Score']],
                         'The Hogwarts First Year Entrance Exam (Unfair)')

# Restored Data (repair_level=1.0)
create_distplot_and_show([unpriv_rp['Score'], priv_rp['Score']],
                         'The Hogwarts First Year Entrance Exam (Restored = 1.0)')

# Restored Data (repair_level=0.8)
create_distplot_and_show([unpriv_rp_2['Score'], priv_rp_2['Score']],
                         'The Hogwarts First Year Entrance Exam (Restored = 0.8)')

# Restored Data (repair_level=0.7)
create_distplot_and_show([unpriv_rp_3['Score'], priv_rp_3['Score']],
                         'The Hogwarts First Year Entrance Exam (Restored = 0.7)')



Training the same model with a restored dataset that used a factor of `1.` makes us lose around 6.5% accuracy. However, we see a great improvement in our fairness metrics.


In [9]:
# Create a `LogisticRegression` classifier and fit it on the restored data
lr_di = LogisticRegression()
lr_di.fit(repaired_train_pd[['Score']], repaired_train_pd['labels'])

# Evaluate the classifier
score = lr_di.score(repaired_test_pd[['Score']], repaired_test_pd['labels'])
di_preds = lr_di.predict(repaired_test_pd[['Score']])
print(f'Accuracy (restoration = 1.): ' + '{:.2f}'.format(score * 100) + ' %')

# Print fairness metrics
fairness_df = calc_fair(lr_di, repaired_test_pd, 'House', 'Gryffindor', 'Slytherin', 'labels')
display(Markdown(fairness_df.to_markdown()))


Accuracy (restoration = 1.): 65.36 %


| Fairness Metrics                                      | Scores                           |
|:------------------------------------------------------|:---------------------------------|
| Chance of receiving the positive class - privileged   | 1.0                              |
| Chance of receiving the positive class - unprivileged | 1.0                              |
| Statistical Parity Ratio (SPR)                        | 1.0                              |
| True Positive Rate - privileged                       | 1.0                              |
| True Positive Rate - unprivileged                     | 1.0                              |
| Equal Opportunity Ratio (EOR)                         | 1.0                              |
| Positive Predictive Value - privileged                | 0.77                             |
| Positive Predictive Value - unprivileged              | 0.36                             |
| Predictive Parity Ratio (PPR)                         | 0.47                             |
| False Positive Rate - privileged                      | 1.0                              |
| False Positive Rate - unprivileged                    | 1.0                              |
| Predictive Equality Ratio (PER)                       | 1.0                              |
| Accuracy - privileged                                 | 0.77                             |
| Accuracy - unprivileged                               | 0.36                             |
| Accuracy Equality Ratio (AER)                         | 0.47                             |
| Equalized Odds                                        | TPR: 1.0 vs 1.0. FPR: 1.0 vs 1.0 |

As the level of restoration decreases...


In [10]:
# Create a `LogisticRegression` classifier and fit it on the restored data
lr_di = LogisticRegression()
lr_di.fit(repaired_train_2_pd[['Score']], repaired_train_2_pd['labels'])

# Evaluate the classifier
score = lr_di.score(repaired_test_2_pd[['Score']], repaired_test_2_pd['labels'])
di_preds = lr_di.predict(repaired_test_2_pd[['Score']])
print(f'Accuracy (restoration = 0.8): ' + '{:.2f}'.format(score * 100) + ' %')

# Print fairness metrics
fairness_df = calc_fair(lr_di, repaired_test_2_pd, 'House', 'Gryffindor', 'Slytherin', 'labels')
display(Markdown(fairness_df.to_markdown()))

Accuracy (restoration = 0.8): 66.07 %


| Fairness Metrics                                      | Scores                            |
|:------------------------------------------------------|:----------------------------------|
| Chance of receiving the positive class - privileged   | 1.0                               |
| Chance of receiving the positive class - unprivileged | 0.98                              |
| Statistical Parity Ratio (SPR)                        | 0.98                              |
| True Positive Rate - privileged                       | 1.0                               |
| True Positive Rate - unprivileged                     | 1.0                               |
| Equal Opportunity Ratio (EOR)                         | 1.0                               |
| Positive Predictive Value - privileged                | 0.77                              |
| Positive Predictive Value - unprivileged              | 0.37                              |
| Predictive Parity Ratio (PPR)                         | 0.48                              |
| False Positive Rate - privileged                      | 1.0                               |
| False Positive Rate - unprivileged                    | 0.96                              |
| Predictive Equality Ratio (PER)                       | 0.96                              |
| Accuracy - privileged                                 | 0.77                              |
| Accuracy - unprivileged                               | 0.39                              |
| Accuracy Equality Ratio (AER)                         | 0.5                               |
| Equalized Odds                                        | TPR: 1.0 vs 1.0. FPR: 1.0 vs 0.96 |

Accuracy increases back.


In [11]:
# Create a `LogisticRegression` classifier and fit it on the restored data
lr_di = LogisticRegression()
lr_di.fit(repaired_train_3_pd[['Score']], repaired_train_3_pd['labels'])

# Evaluate the classifier
score = lr_di.score(repaired_test_3_pd[['Score']], repaired_test_3_pd['labels'])
di_preds = lr_di.predict(repaired_test_3_pd[['Score']])
print(f'Accuracy (restoration = 0.7): ' + '{:.2f}'.format(score * 100) + ' %')

# Print fairness metrics
fairness_df = calc_fair(lr_di, repaired_test_3_pd, 'House', 'Gryffindor', 'Slytherin', 'labels')
display(Markdown(fairness_df.to_markdown()))

Accuracy (restoration = 0.7): 68.93 %


| Fairness Metrics                                      | Scores                             |
|:------------------------------------------------------|:-----------------------------------|
| Chance of receiving the positive class - privileged   | 1.0                                |
| Chance of receiving the positive class - unprivileged | 0.85                               |
| Statistical Parity Ratio (SPR)                        | 0.85                               |
| True Positive Rate - privileged                       | 1.0                                |
| True Positive Rate - unprivileged                     | 0.97                               |
| Equal Opportunity Ratio (EOR)                         | 0.97                               |
| Positive Predictive Value - privileged                | 0.77                               |
| Positive Predictive Value - unprivileged              | 0.41                               |
| Predictive Parity Ratio (PPR)                         | 0.53                               |
| False Positive Rate - privileged                      | 1.0                                |
| False Positive Rate - unprivileged                    | 0.78                               |
| Predictive Equality Ratio (PER)                       | 0.78                               |
| Accuracy - privileged                                 | 0.77                               |
| Accuracy - unprivileged                               | 0.49                               |
| Accuracy Equality Ratio (AER)                         | 0.63                               |
| Equalized Odds                                        | TPR: 1.0 vs 0.97. FPR: 1.0 vs 0.78 |

Accuracy is compromised when we optimize for Fairness. However, a marginal loss in performance is (according to many researchers in the field) an **_acceptable evil_** when the benefit gained is less discrimination. Nonetheless, the discussion on how much accuracy we can sacrifice in the name of Fairness is an ongoing debate.

---

Return to the [castle](https://github.com/Nkluge-correa/TeenyTinyCastle).